In [6]:
# Importando bibliotecas utilizadas
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from math import sqrt, pow
from datetime import datetime
from time import sleep

In [1]:
#Distancia Euclidiana
# Mudei a forma como é calculado para poder utilizar vetorização
def distancia_euclidiana(x1, y1, x2, y2):
    X = x1 - x2
    Y = y1 - y2
    
    X = X*X
    Y = Y*Y
    
    distancia = np.sqrt(X+Y)
    return distancia

In [2]:
def dist_matrix(df):
    lista = []
    for i, row in df.iterrows():
        distancias = distancia_euclidiana(row['d1'], row['d2'], df['d1'].values, df['d2'].values)
        
        lista.append(distancias)
    
    matrix = np.array(lista)
    np.fill_diagonal(matrix, float('inf'))
    
    
    return matrix

In [3]:
def agrupar_clusters(clusters, objetos, numero_de_objetos):
    obj1, obj2 =  objetos
    cluster1 = clusters[obj1]
    cluster2 = clusters[obj2]

    
    if cluster1 < cluster2:
        menor_cluster = cluster1
        maior_cluster = cluster2
    else:
        menor_cluster = cluster2
        maior_cluster = cluster1
        
    pos_menor = np.array(np.where(clusters == menor_cluster))
    pos_maior = np.array(np.where(clusters == maior_cluster))

    clusters[clusters == maior_cluster] = menor_cluster

In [4]:
def mst_single_link(matriz, numero_de_objetos):
    # Matriz com objetos e distancia até eles
    # obj1 obj2 dist
    ordem_de_agrupamento = np.zeros((numero_de_objetos-1,3))
    
    # Quais objetos já foram juntados
    conectados = np.zeros(numero_de_objetos)
    
    #Distância até cada objeto que é possível alcançar
    #Seria a distância entres os objetos que já temos como alcançar do algoritmo de Prim
    #Começa todos com infinito
    dist_conhecidas = np.zeros(numero_de_objetos)
    dist_conhecidas[:] = float('inf')
    
    # obj 1 e 2 serão os pontos
    # dist_atual é a menor distância de cada iteração
    obj2 = dist_atual = -1
    obj1 = 0
    
    for i in range(numero_de_objetos-1):
        conectados[obj1] = 1
        dist_atual = float('inf')
        for j in range(numero_de_objetos):
            if conectados[j] == 1:
                continue
                
            dist = matriz[obj1][j]
            if dist_conhecidas[j] > dist:
                dist_conhecidas[j] = dist
            
            
            if dist_conhecidas[j] < dist_atual:
                dist_atual = dist
                obj2 = j
        
        ordem_de_agrupamento[i][0] = obj1
        ordem_de_agrupamento[i][1] = obj2
        ordem_de_agrupamento[i][2] = dist_atual
        obj1 = obj2
        
    return ordem_de_agrupamento[np.argsort(ordem_de_agrupamento[:,2])]
        

In [5]:
def single_link(dataset, k_min, k_max, nome):
    numero_de_objetos = len(dataset)
    matriz = dist_matrix(dataset)
    clusters = np.arange(numero_de_objetos, dtype=int)
    
    qtd_clusters = numero_de_objetos
    ordem_de_agrupamento = mst_single_link(matriz, numero_de_objetos)
    
    for i in range(numero_de_objetos-k_min):
        obj1 = int(ordem_de_agrupamento[i][0])
        obj2 = int(ordem_de_agrupamento[i][1])
        agrupar_clusters(clusters, [obj1,obj2],numero_de_objetos)
    
    return clusters

## Teste

In [7]:
df3 = pd.read_csv('datasets/monkey.txt', sep='\t')
df3.columns = ['identificador','d1', 'd2']
df3.head()

,identificador,d1,d2
0,monkeyc1g1s1,8.809783,7.611147
1,monkeyc1g1s2,4.110747,11.103186
2,monkeyc1g1s3,4.114710,11.039587
3,monkeyc1g1s4,3.154736,6.743244
4,monkeyc1g1s5,5.972931,7.537982


In [8]:
%%timeit
clusters_df3 = single_link(df3, 5, 7, 'teste')

15.8 s ± 1.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
